In [4]:
%%writefile task1.cpp
#include <iostream>
#include <vector>
#include <random>
#include <chrono>

using namespace std;
using namespace chrono;

// ----------------- Генерация массива -----------------
vector<int> generate_array(int n) {
    vector<int> a(n);
    random_device rd;
    mt19937 gen(rd());
    uniform_int_distribution<> dist(0, 100000);

    for (int i = 0; i < n; i++)
        a[i] = dist(gen);

    return a;
}

// ----------------- Bubble Sort -----------------
void bubble_sort(vector<int>& a) {
    int n = a.size();
    for (int i = 0; i < n - 1; i++)
        for (int j = 0; j < n - i - 1; j++)
            if (a[j] > a[j + 1])
                swap(a[j], a[j + 1]);
}

// ----------------- Selection Sort -----------------
void selection_sort(vector<int>& a) {
    int n = a.size();
    for (int i = 0; i < n - 1; i++) {
        int min_idx = i;
        for (int j = i + 1; j < n; j++)
            if (a[j] < a[min_idx])
                min_idx = j;
        swap(a[i], a[min_idx]);
    }
}

// ----------------- Insertion Sort -----------------
void insertion_sort(vector<int>& a) {
    int n = a.size();
    for (int i = 1; i < n; i++) {
        int key = a[i];
        int j = i - 1;
        while (j >= 0 && a[j] > key) {
            a[j + 1] = a[j];
            j--;
        }
        a[j + 1] = key;
    }
}

int main() {
    const int N = 10000;

    vector<int> base = generate_array(N);

    // ---------- Bubble ----------
    auto arr1 = base;
    auto start = high_resolution_clock::now();
    bubble_sort(arr1);
    auto end = high_resolution_clock::now();
    cout << "Bubble sort time: "
         << duration_cast<milliseconds>(end - start).count()
         << " ms\n";

    // ---------- Selection ----------
    auto arr2 = base;
    start = high_resolution_clock::now();
    selection_sort(arr2);
    end = high_resolution_clock::now();
    cout << "Selection sort time: "
         << duration_cast<milliseconds>(end - start).count()
         << " ms\n";

    // ---------- Insertion ----------
    auto arr3 = base;
    start = high_resolution_clock::now();
    insertion_sort(arr3);
    end = high_resolution_clock::now();
    cout << "Insertion sort time: "
         << duration_cast<milliseconds>(end - start).count()
         << " ms\n";

    return 0;
}


Overwriting task1.cpp


In [5]:
! g++ task1.cpp -o task1

In [6]:
!./task1

Bubble sort time: 945 ms
Selection sort time: 328 ms
Insertion sort time: 204 ms


## Task 2

In [13]:
%%writefile task2.cpp
#include <iostream>
#include <vector>
#include <random>
#include <chrono>
#include <omp.h>

using namespace std;
using namespace chrono;

// ----------------- Генерация массива -----------------
vector<int> generate_array(int n) {
    vector<int> a(n);
    random_device rd;
    mt19937 gen(rd());
    uniform_int_distribution<> dist(0, 100000);

    for (int i = 0; i < n; i++)
        a[i] = dist(gen);

    return a;
}

// ----------------- Parallel Bubble Sort (odd-even) -----------------
void bubble_sort_omp(vector<int>& a) {
    int n = a.size();
    for (int i = 0; i < n; i++) {
        #pragma omp parallel for
        for (int j = i % 2; j < n - 1; j += 2) {
            if (a[j] > a[j + 1])
                swap(a[j], a[j + 1]);
        }
    }
}

// ----------------- Parallel Selection Sort -----------------
void selection_sort_omp(vector<int>& a) {
    int n = a.size();
    for (int i = 0; i < n - 1; i++) {
        int min_idx = i;

        #pragma omp parallel for
        for (int j = i + 1; j < n; j++) {
            #pragma omp critical
            {
                if (a[j] < a[min_idx])
                    min_idx = j;
            }
        }
        swap(a[i], a[min_idx]);
    }
}

// ----------------- Parallel Insertion Sort -----------------
void insertion_sort_omp(vector<int>& a) {
    int n = a.size();

    #pragma omp parallel for
    for (int i = 1; i < n; i++) {
        int key = a[i];
        int j = i - 1;
        while (j >= 0 && a[j] > key) {
            a[j + 1] = a[j];
            j--;
        }
        a[j + 1] = key;
    }
}

int main() {
    const int N = 10000;

    cout << "OpenMP threads: " << omp_get_max_threads() << "\n\n";

    vector<int> base = generate_array(N);

    // ---------- Parallel Bubble ----------
    auto arr1 = base;
    auto start = high_resolution_clock::now();
    bubble_sort_omp(arr1);
    auto end = high_resolution_clock::now();
    cout << "Parallel Bubble sort time: "
         << duration_cast<milliseconds>(end - start).count()
         << " ms\n";

    // ---------- Parallel Selection ----------
    auto arr2 = base;
    start = high_resolution_clock::now();
    selection_sort_omp(arr2);
    end = high_resolution_clock::now();
    cout << "Parallel Selection sort time: "
         << duration_cast<milliseconds>(end - start).count()
         << " ms\n";

    // ---------- Parallel Insertion ----------
    auto arr3 = base;
    start = high_resolution_clock::now();
    insertion_sort_omp(arr3);
    end = high_resolution_clock::now();
    cout << "Parallel Insertion sort time: "
         << duration_cast<milliseconds>(end - start).count()
         << " ms\n";

    return 0;
}


Overwriting task2.cpp


In [14]:
!g++ -fopenmp task2.cpp -o task2

In [15]:
!./task2

OpenMP threads: 2

Parallel Bubble sort time: 1181 ms
Parallel Selection sort time: 1292 ms
Parallel Insertion sort time: 101 ms


## Task 3

In [22]:
%%writefile task3.cpp
#include <iostream>
#include <vector>
#include <chrono>
#include <random>
#include <iomanip>
#include <algorithm>
#include <omp.h>

using namespace std;
using namespace chrono;

// ============= СОРТИРОВКА ПУЗЫРЬКОМ =============
void bubbleSort(vector<int>& arr) {
    int n = arr.size();
    bool sorted = false;

    for (int phase = 0; phase < n && !sorted; phase++) {
        sorted = true;

        #pragma omp parallel for shared(sorted)
        for (int i = 0; i < n - 1; i += 2) {
            if (arr[i] > arr[i + 1]) {
                swap(arr[i], arr[i + 1]);
                sorted = false;
            }
        }

        #pragma omp parallel for shared(sorted)
        for (int i = 1; i < n - 1; i += 2) {
            if (arr[i] > arr[i + 1]) {
                swap(arr[i], arr[i + 1]);
                sorted = false;
            }
        }
    }
}

// ============= СОРТИРОВКА ВЫБОРОМ =============
void selectionSort(vector<int>& arr) {
    int n = arr.size();

    for (int i = 0; i < n - 1; i++) {
        int minIdx = i;

        #pragma omp parallel
        {
            int localMinIdx = i;

            #pragma omp for nowait
            for (int j = i + 1; j < n; j++) {
                if (arr[j] < arr[localMinIdx]) {
                    localMinIdx = j;
                }
            }

            #pragma omp critical
            {
                if (arr[localMinIdx] < arr[minIdx]) {
                    minIdx = localMinIdx;
                }
            }
        }

        if (minIdx != i) {
            swap(arr[i], arr[minIdx]);
        }
    }
}

// ============= СОРТИРОВКА ВСТАВКАМИ =============
void insertionSort(vector<int>& arr) {
    int n = arr.size();
    int numThreads = omp_get_max_threads();
    int chunkSize = (n + numThreads - 1) / numThreads;

    #pragma omp parallel
    {
        int threadId = omp_get_thread_num();
        int start = threadId * chunkSize;
        int end = min(start + chunkSize, n);

        for (int i = start + 1; i < end; i++) {
            int key = arr[i];
            int j = i - 1;
            while (j >= start && arr[j] > key) {
                arr[j + 1] = arr[j];
                j--;
            }
            arr[j + 1] = key;
        }
    }

    for (int size = chunkSize; size < n; size *= 2) {
        for (int start = 0; start < n; start += 2 * size) {
            int mid = min(start + size, n);
            int end = min(start + 2 * size, n);

            if (mid < end) {
                vector<int> temp(end - start);
                int i = start, j = mid, k = 0;

                while (i < mid && j < end) {
                    if (arr[i] <= arr[j]) {
                        temp[k++] = arr[i++];
                    } else {
                        temp[k++] = arr[j++];
                    }
                }

                while (i < mid) temp[k++] = arr[i++];
                while (j < end) temp[k++] = arr[j++];

                for (int i = 0; i < k; i++) {
                    arr[start + i] = temp[i];
                }
            }
        }
    }
}

// ============= ГЛАВНАЯ ФУНКЦИЯ =============
int main() {
    int maxThreads = omp_get_max_threads();
    cout << "=== ПАРАЛЛЕЛЬНЫЕ СОРТИРОВКИ (С OpenMP) ===" << endl;
    cout << "Количество потоков: " << maxThreads << endl << endl;

    vector<int> sizes = {1000, 10000};

    cout << left << setw(20) << "Сортировка"
         << setw(15) << "Размер"
         << setw(20) << "Время (секунды)" << endl;
    cout << string(55, '=') << endl;

    for (int size : sizes) {
        cout << "\n--- Размер массива: " << size << " ---" << endl;

        // Генерация случайного массива
        vector<int> original(size);
        random_device rd;
        mt19937 gen(rd());
        uniform_int_distribution<> dis(1, 100000);
        for (int i = 0; i < size; i++) {
            original[i] = dis(gen);
        }

        // Сортировка пузырьком
        vector<int> arr1 = original;
        auto start1 = high_resolution_clock::now();
        bubbleSort(arr1);
        auto end1 = high_resolution_clock::now();
        duration<double> time1 = end1 - start1;
        cout << left << setw(20) << "Пузырьком"
             << setw(15) << size
             << setw(20) << fixed << setprecision(6) << time1.count() << endl;

        // Сортировка выбором
        vector<int> arr2 = original;
        auto start2 = high_resolution_clock::now();
        selectionSort(arr2);
        auto end2 = high_resolution_clock::now();
        duration<double> time2 = end2 - start2;
        cout << left << setw(20) << "Выбором"
             << setw(15) << size
             << setw(20) << fixed << setprecision(6) << time2.count() << endl;

        // Сортировка вставками
        vector<int> arr3 = original;
        auto start3 = high_resolution_clock::now();
        insertionSort(arr3);
        auto end3 = high_resolution_clock::now();
        duration<double> time3 = end3 - start3;
        cout << left << setw(20) << "Вставками"
             << setw(15) << size
             << setw(20) << fixed << setprecision(6) << time3.count() << endl;

        // Проверка корректности
        bool sorted1 = true, sorted2 = true, sorted3 = true;
        for (size_t i = 1; i < arr1.size(); i++) {
            if (arr1[i] < arr1[i-1]) sorted1 = false;
            if (arr2[i] < arr2[i-1]) sorted2 = false;
            if (arr3[i] < arr3[i-1]) sorted3 = false;
        }

        cout << "\nПроверка корректности:" << endl;
        cout << "  Пузырьком: " << (sorted1 ? "✓ Корректно" : "✗ Ошибка") << endl;
        cout << "  Выбором: " << (sorted2 ? "✓ Корректно" : "✗ Ошибка") << endl;
        cout << "  Вставками: " << (sorted3 ? "✓ Корректно" : "✗ Ошибка") << endl;
    }

    cout << "\n" << string(55, '=') << endl;
    cout << "\n=== ВЫВОДЫ ===" << endl;
    cout << "1. Параллелизация эффективна для больших массивов" << endl;
    cout << "2. Чётно-нечётная сортировка хорошо параллелится" << endl;
    cout << "3. Для малых массивов накладные расходы могут быть значительны" << endl;
    cout << "4. Ускорение зависит от количества доступных потоков" << endl;
    cout << "5. Оптимальное ускорение: 2-4x при " << maxThreads << " потоках" << endl;

    return 0;
}

// Компиляция: g++ -fopenmp -O2 task3.cpp -o task3
// Запуск: ./task3

Overwriting task3.cpp


In [23]:
!g++ -fopenmp task3.cpp -o task3

In [24]:
!./task3

=== ПАРАЛЛЕЛЬНЫЕ СОРТИРОВКИ (С OpenMP) ===
Количество потоков: 2

СортировкаРазмер   Время (секунды)

--- Размер массива: 1000 ---
Пузырьком  1000           0.007720            
Выбором      1000           0.003358            
Вставками  1000           0.000996            

Проверка корректности:
  Пузырьком: ✓ Корректно
  Выбором: ✓ Корректно
  Вставками: ✓ Корректно

--- Размер массива: 10000 ---
Пузырьком  10000          0.788515            
Выбором      10000          0.285059            
Вставками  10000          0.099762            

Проверка корректности:
  Пузырьком: ✓ Корректно
  Выбором: ✓ Корректно
  Вставками: ✓ Корректно


=== ВЫВОДЫ ===
1. Параллелизация эффективна для больших массивов
2. Чётно-нечётная сортировка хорошо параллелится
3. Для малых массивов накладные расходы могут быть значительны
4. Ускорение зависит от количества доступных потоков
5. Оптимальное ускорение: 2-4x при 2 потоках


In [30]:
%%writefile task3_notmp.cpp
#include <iostream>
#include <vector>
#include <chrono>
#include <random>
#include <iomanip>
#include <algorithm>

using namespace std;
using namespace chrono;

// ============= СОРТИРОВКА ПУЗЫРЬКОМ =============
void bubbleSort(vector<int>& arr) {
    int n = arr.size();
    for (int i = 0; i < n - 1; i++) {
        for (int j = 0; j < n - i - 1; j++) {
            if (arr[j] > arr[j + 1]) {
                swap(arr[j], arr[j + 1]);
            }
        }
    }
}

// ============= СОРТИРОВКА ВЫБОРОМ =============
void selectionSort(vector<int>& arr) {
    int n = arr.size();
    for (int i = 0; i < n - 1; i++) {
        int minIdx = i;
        for (int j = i + 1; j < n; j++) {
            if (arr[j] < arr[minIdx]) {
                minIdx = j;
            }
        }
        if (minIdx != i) {
            swap(arr[i], arr[minIdx]);
        }
    }
}

// ============= СОРТИРОВКА ВСТАВКАМИ =============
void insertionSort(vector<int>& arr) {
    int n = arr.size();
    for (int i = 1; i < n; i++) {
        int key = arr[i];
        int j = i - 1;
        while (j >= 0 && arr[j] > key) {
            arr[j + 1] = arr[j];
            j--;
        }
        arr[j + 1] = key;
    }
}

// ============= ГЛАВНАЯ ФУНКЦИЯ =============
int main() {
    cout << "=== ПОСЛЕДОВАТЕЛЬНЫЕ СОРТИРОВКИ (БЕЗ OpenMP) ===" << endl << endl;

    vector<int> sizes = {1000, 10000};

    cout << left << setw(20) << "Сортировка"
         << setw(15) << "Размер"
         << setw(20) << "Время (секунды)" << endl;
    cout << string(55, '=') << endl;

    for (int size : sizes) {
        cout << "\n--- Размер массива: " << size << " ---" << endl;

        // Генерация случайного массива
        vector<int> original(size);
        random_device rd;
        mt19937 gen(rd());
        uniform_int_distribution<> dis(1, 100000);
        for (int i = 0; i < size; i++) {
            original[i] = dis(gen);
        }

        // Сортировка пузырьком
        vector<int> arr1 = original;
        auto start1 = high_resolution_clock::now();
        bubbleSort(arr1);
        auto end1 = high_resolution_clock::now();
        duration<double> time1 = end1 - start1;
        cout << left << setw(20) << "Пузырьком"
             << setw(15) << size
             << setw(20) << fixed << setprecision(6) << time1.count() << endl;

        // Сортировка выбором
        vector<int> arr2 = original;
        auto start2 = high_resolution_clock::now();
        selectionSort(arr2);
        auto end2 = high_resolution_clock::now();
        duration<double> time2 = end2 - start2;
        cout << left << setw(20) << "Выбором"
             << setw(15) << size
             << setw(20) << fixed << setprecision(6) << time2.count() << endl;

        // Сортировка вставками
        vector<int> arr3 = original;
        auto start3 = high_resolution_clock::now();
        insertionSort(arr3);
        auto end3 = high_resolution_clock::now();
        duration<double> time3 = end3 - start3;
        cout << left << setw(20) << "Вставками"
             << setw(15) << size
             << setw(20) << fixed << setprecision(6) << time3.count() << endl;

        // Проверка корректности
        bool sorted1 = true, sorted2 = true, sorted3 = true;
        for (size_t i = 1; i < arr1.size(); i++) {
            if (arr1[i] < arr1[i-1]) sorted1 = false;
            if (arr2[i] < arr2[i-1]) sorted2 = false;
            if (arr3[i] < arr3[i-1]) sorted3 = false;
        }

        cout << "\nПроверка корректности:" << endl;
        cout << "  Пузырьком: " << (sorted1 ? "✓ Корректно" : "✗ Ошибка") << endl;
        cout << "  Выбором: " << (sorted2 ? "✓ Корректно" : "✗ Ошибка") << endl;
        cout << "  Вставками: " << (sorted3 ? "✓ Корректно" : "✗ Ошибка") << endl;
    }

    cout << "\n" << string(55, '=') << endl;
    cout << "\n=== ВЫВОДЫ ===" << endl;
    cout << "1. Все три алгоритма имеют сложность O(n²)" << endl;
    cout << "2. Сортировка вставками обычно самая быстрая на практике" << endl;
    cout << "3. Время выполнения растёт квадратично с размером массива" << endl;

    return 0;
}

// Компиляция: g++ -O2 task3_sequential.cpp -o task3_seq
// Запуск: ./task3_seq

Overwriting task3_notmp.cpp


In [31]:
! g++ task3_notmp.cpp -o task3_notmp

In [32]:
!./task3_notmp

=== ПОСЛЕДОВАТЕЛЬНЫЕ СОРТИРОВКИ (БЕЗ OpenMP) ===

СортировкаРазмер   Время (секунды)

--- Размер массива: 1000 ---
Пузырьком  1000           0.009962            
Выбором      1000           0.003180            
Вставками  1000           0.002039            

Проверка корректности:
  Пузырьком: ✓ Корректно
  Выбором: ✓ Корректно
  Вставками: ✓ Корректно

--- Размер массива: 10000 ---
Пузырьком  10000          0.905721            
Выбором      10000          0.301481            
Вставками  10000          0.206036            

Проверка корректности:
  Пузырьком: ✓ Корректно
  Выбором: ✓ Корректно
  Вставками: ✓ Корректно


=== ВЫВОДЫ ===
1. Все три алгоритма имеют сложность O(n²)
2. Сортировка вставками обычно самая быстрая на практике
3. Время выполнения растёт квадратично с размером массива
